In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = "/content/heart_2022_with_nans.csv"
data=pd.read_csv(path)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      445132 non-null  object 
 1   Sex                        445132 non-null  object 
 2   GeneralHealth              443934 non-null  object 
 3   PhysicalHealthDays         434205 non-null  float64
 4   MentalHealthDays           436065 non-null  float64
 5   LastCheckupTime            436824 non-null  object 
 6   PhysicalActivities         444039 non-null  object 
 7   SleepHours                 439679 non-null  float64
 8   RemovedTeeth               433772 non-null  object 
 9   HadHeartAttack             442067 non-null  object 
 10  HadAngina                  440727 non-null  object 
 11  HadStroke                  443575 non-null  object 
 12  HadAsthma                  443359 non-null  object 
 13  HadSkinCancer              44

In [4]:
# pd.get_dummies(data)

In [5]:
catlist=[]
numlist=[]
for coulmn in data.columns:
  if data[coulmn].dtype == 'object':
    catlist.append(coulmn)
  else:
    numlist.append(coulmn)
catlist


['State',
 'Sex',
 'GeneralHealth',
 'LastCheckupTime',
 'PhysicalActivities',
 'RemovedTeeth',
 'HadHeartAttack',
 'HadAngina',
 'HadStroke',
 'HadAsthma',
 'HadSkinCancer',
 'HadCOPD',
 'HadDepressiveDisorder',
 'HadKidneyDisease',
 'HadArthritis',
 'HadDiabetes',
 'DeafOrHardOfHearing',
 'BlindOrVisionDifficulty',
 'DifficultyConcentrating',
 'DifficultyWalking',
 'DifficultyDressingBathing',
 'DifficultyErrands',
 'SmokerStatus',
 'ECigaretteUsage',
 'ChestScan',
 'RaceEthnicityCategory',
 'AgeCategory',
 'AlcoholDrinkers',
 'HIVTesting',
 'FluVaxLast12',
 'PneumoVaxEver',
 'TetanusLast10Tdap',
 'HighRiskLastYear',
 'CovidPos']

In [6]:
from sklearn.preprocessing import OrdinalEncoder
orde=OrdinalEncoder()
orde.fit(data[catlist])
data[catlist]=orde.transform(data[catlist])




In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State                      445132 non-null  float64
 1   Sex                        445132 non-null  float64
 2   GeneralHealth              443934 non-null  float64
 3   PhysicalHealthDays         434205 non-null  float64
 4   MentalHealthDays           436065 non-null  float64
 5   LastCheckupTime            436824 non-null  float64
 6   PhysicalActivities         444039 non-null  float64
 7   SleepHours                 439679 non-null  float64
 8   RemovedTeeth               433772 non-null  float64
 9   HadHeartAttack             442067 non-null  float64
 10  HadAngina                  440727 non-null  float64
 11  HadStroke                  443575 non-null  float64
 12  HadAsthma                  443359 non-null  float64
 13  HadSkinCancer              44

In [ ]:
##cheack if data has duublicate
data.duplicated().sum()

np.int64(157)

In [11]:
data=data.drop_duplicates()
data.duplicated().sum()

np.int64(0)

In [ ]:
 data.drop(['RemovedTeeth'],axis =1, inplace=True)

In [ ]:
# data.drop(['HadSkinCancer'],axis =1, inplace=True)

In [ ]:
# data.drop(['DeafOrHardOfHearing'],axis =1, inplace=True)

In [ ]:
# data.drop(['BlindOrVisionDifficulty'],axis =1, inplace=True)

In [ ]:
# data.drop(['DifficultyConcentrating'],axis =1, inplace=True)

In [ ]:
# data.drop(['DifficultyDressingBathing'],axis =1, inplace=True)

In [ ]:
# data.drop(['HeightInMeters'],axis =1, inplace=True)

In [ ]:
# data.drop(['HadCOPD'],axis =1, inplace=True)

In [ ]:
# data.drop(['HadAsthma'],axis =1, inplace=True)

In [ ]:
# data.drop(['HadKidneyDisease'],axis =1, inplace=True)

In [ ]:
## cheack if data has nulls
data.isna().sum()

,0
State,0
Sex,0
GeneralHealth,1198
PhysicalHealthDays,10927
MentalHealthDays,9067
LastCheckupTime,8308
PhysicalActivities,1093
SleepHours,5453
RemovedTeeth,11360
HadHeartAttack,3065


In [ ]:
# hadel nulls in WeightInKilograms by mod
data['WeightInKilograms'].fillna(data['WeightInKilograms'].mode()[0], inplace=True)

<ipython-input-135-28154643e902>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['WeightInKilograms'].fillna(data['WeightInKilograms'].mode()[0], inplace=True)


In [ ]:
data.isna().sum()

,0
State,0
Sex,0
GeneralHealth,1198
PhysicalHealthDays,10927
MentalHealthDays,9067
LastCheckupTime,8308
PhysicalActivities,1093
SleepHours,5453
RemovedTeeth,11360
HadHeartAttack,3065


In [ ]:
#fill all data with midian
data.fillna(data.ffill(), inplace=True)

In [ ]:
#handel nulls with backwards
data.fillna(method='bfill', inplace=True)


<ipython-input-138-d6ed7faa274d>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


In [ ]:
data.isna().sum()

,0
State,0
Sex,0
GeneralHealth,0
PhysicalHealthDays,0
MentalHealthDays,0
LastCheckupTime,0
PhysicalActivities,0
SleepHours,0
RemovedTeeth,0
HadHeartAttack,0


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.drop(['HadHeartAttack','GeneralHealth'], axis=1), data['HadHeartAttack'], test_size=0.02, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
accuracy_score(y_test,y_pred)


0.9421543300011233

In [ ]:

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97      8362
         1.0       0.58      0.17      0.26       541

    accuracy                           0.94      8903
   macro avg       0.77      0.58      0.62      8903
weighted avg       0.93      0.94      0.93      8903



In [ ]:
data.head()

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,8.0,3.0,0.0,...,1.70,90.72,27.44,0.0,0.0,1.0,0.0,2.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,3.0,0.0,6.0,3.0,0.0,...,1.60,68.04,26.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,2.0,3.0,3.0,1.0,5.0,3.0,0.0,...,1.57,63.50,25.61,0.0,0.0,0.0,0.0,1.0,0.0,2.0
3,0.0,0.0,0.0,0.0,0.0,3.0,1.0,7.0,3.0,0.0,...,1.65,63.50,23.30,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,2.0,0.0,3.0,1.0,9.0,3.0,0.0,...,1.57,53.98,21.77,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
data.to_csv('exported_heart_data.csv', index=False)